In [1]:
import nltk
import re
import spacy
import numpy as np
from tqdm import tqdm
from nltk.corpus import stopwords  
from sklearn.model_selection import PredefinedSplit
import sklearn.datasets
import sklearn.metrics
import pandas as pd
import pickle
import warnings
import matplotlib.pyplot as plt

# Language Model Results Table

Here we create overview tables of the language model results as a LaTeX table

In [2]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

path = "../Language Model-Results/"
path2 = "../../../Evaluation/Baseline-Results/"

bertbase_models_min2 = {"bert-base-german-cased-baseline-unfrozen-RP-Crowd-2-folds_best.pickle":0, "bert-base-german-cased-double-head-unfrozen-RP-Crowd-2-folds-a0.1_best.pickle":0,"bert-base-german-cased-double-head-unfrozen-RP-Crowd-2-folds-a0.5_best.pickle":0, "bert-base-german-cased-double-head-unfrozen-RP-Crowd-2-folds-a0.9_best.pickle":0}
bertbase_models_min3 = {"bert-base-german-cased-baseline-unfrozen-RP-Crowd-3-folds_best.pickle":0,"bert-base-german-cased-double-head-unfrozen-RP-Crowd-3-folds-a0.1_best.pickle":0, "bert-base-german-cased-double-head-unfrozen-RP-Crowd-3-folds-a0.5_best.pickle":0, "bert-base-german-cased-double-head-unfrozen-RP-Crowd-3-folds-a0.9_best.pickle":0}
bertbase_models_mod = {"bert-base-german-cased-baseline-unfrozen-RP-Mod-folds_best.pickle":0, "bert-base-german-cased-double-head-unfrozen-RP-Mod-folds-a0.1_best.pickle":0,"bert-base-german-cased-double-head-unfrozen-RP-Mod-folds-a0.5_best.pickle":0, "bert-base-german-cased-double-head-unfrozen-RP-Mod-folds-a0.9_best.pickle":0}

berthate_models_min2 = {"bert-base-german-cased-hatespeech-GermEval18Coarse-baseline-unfrozen-RP-Crowd-2-folds_best.pickle":0, "bert-base-german-cased-hatespeech-GermEval18Coarse-double-head-unfrozen-RP-Crowd-2-folds-a0.1_best.pickle":0,"bert-base-german-cased-hatespeech-GermEval18Coarse-double-head-unfrozen-RP-Crowd-2-folds-a0.5_best.pickle":0, "bert-base-german-cased-hatespeech-GermEval18Coarse-double-head-unfrozen-RP-Crowd-2-folds-a0.9_best.pickle":0}
berthate_models_min3 = {"bert-base-german-cased-hatespeech-GermEval18Coarse-baseline-unfrozen-RP-Crowd-3-folds_best.pickle":0, "bert-base-german-cased-hatespeech-GermEval18Coarse-double-head-unfrozen-RP-Crowd-3-folds-a0.1_best.pickle":0,"bert-base-german-cased-hatespeech-GermEval18Coarse-double-head-unfrozen-RP-Crowd-3-folds-a0.5_best.pickle":0, "bert-base-german-cased-hatespeech-GermEval18Coarse-double-head-unfrozen-RP-Crowd-3-folds-a0.9_best.pickle":0}
berthate_models_mod = {"bert-base-german-cased-hatespeech-GermEval18Coarse-baseline-unfrozen-RP-Mod-folds_best.pickle":0, "bert-base-german-cased-hatespeech-GermEval18Coarse-double-head-unfrozen-RP-Mod-folds-a0.1_best.pickle":0,"bert-base-german-cased-hatespeech-GermEval18Coarse-double-head-unfrozen-RP-Mod-folds-a0.5_best.pickle":0, "bert-base-german-cased-hatespeech-GermEval18Coarse-double-head-unfrozen-RP-Mod-folds-a0.9_best.pickle":0}

gbert_models_min2 = {"gbert-base-baseline-unfrozen-RP-Crowd-2-folds_best.pickle":0, "gbert-base-double-head-unfrozen-RP-Crowd-2-folds-a0.1_best.pickle":0,"gbert-base-double-head-unfrozen-RP-Crowd-2-folds-a0.5_best.pickle":0, "gbert-base-double-head-unfrozen-RP-Crowd-2-folds-a0.9_best.pickle":0}
gbert_models_min3 = {"gbert-base-baseline-unfrozen-RP-Crowd-3-folds_best.pickle":0, "gbert-base-double-head-unfrozen-RP-Crowd-3-folds-a0.1_best.pickle":0,"gbert-base-double-head-unfrozen-RP-Crowd-3-folds-a0.5_best.pickle":0, "gbert-base-double-head-unfrozen-RP-Crowd-3-folds-a0.9_best.pickle":0}
gbert_models_mod = {"gbert-base-baseline-unfrozen-RP-Mod-folds_best.pickle":0, "gbert-base-double-head-unfrozen-RP-Mod-folds-a0.1_best.pickle":0,"gbert-base-double-head-unfrozen-RP-Mod-folds-a0.5_best.pickle":0, "gbert-base-double-head-unfrozen-RP-Mod-folds-a0.9_best.pickle":0}


In [3]:
import pandas as pd
import numpy as np

## ACCURACY TABLE

table_header = '\\begin{tabular}{ llllll }   \n\
\\multicolumn{2}{c}{}&\\multicolumn{1}{c}{Single-Task}& \\multicolumn{3}{ c }{Multi-Task} \\\ \n\
& & & $\\alpha$ =0.1 & $\\alpha$ = 0.5 & $\\alpha$ = 0.9\\\ \\hline \n\
'

## BERT BASE MODEL

bertbase = '\multirow{3}{*}{\\texttt{BERT$_{Base}$}}'
bertbaserp2 = '&\\texttt{RP-Crowd-2}'

for key in bertbase_models_min2:
    data = pickle.load( open( path+key, "rb" ))
    bertbaserp2 += "&"+ str(round(accuracy_score(data["y_trues"],data["decisions"]),3))
bertbaserp2 +=  "\\\\ \n"

bertbaserp3 = '&\\texttt{RP-Crowd-3}'
for key in bertbase_models_min3:
    data = pickle.load( open( path+key, "rb" ))
    bertbaserp3 += "&"+ str(round(accuracy_score(data["y_trues"],data["decisions"]),3))
bertbaserp3 +=  "\\\\ \n"


bertbasemod = '&\\texttt{RP-Mod}'
for key in bertbase_models_mod:
    data = pickle.load( open( path+key, "rb" ))
    bertbasemod += "&"+ str(round(accuracy_score(data["y_trues"],data["decisions"]),3))
bertbasemod +=  "\\\\ \hline \n"

############### BERT HATESPEECH MODEl

berthate = '\multirow{3}{*}{\\texttt{BERT$_{Hate}$}}'
berthate2 = '&\\texttt{RP-Crowd-2}'

for key in berthate_models_min2:
    data = pickle.load( open( path+key, "rb" ))
    berthate2 += "&"+ str(round(accuracy_score(data["y_trues"],data["decisions"]),3))
berthate2 +=  "\\\\ \n"

berthate3 = '&\\texttt{RP-Crowd-3}'
for key in berthate_models_min3:
    data = pickle.load( open( path+key, "rb" ))
    berthate3 += "&"+ str(round(accuracy_score(data["y_trues"],data["decisions"]),3))
berthate3 +=  "\\\\\n"

berthatemod = '&\\texttt{RP-Mod}'
for key in berthate_models_mod:
    data = pickle.load( open( path+key, "rb" ))
    berthatemod += "&"+ str(round(accuracy_score(data["y_trues"],data["decisions"]),3))
berthatemod +=  "\\\\ \hline \n"

############### BERT GBERT MODEl

gbert= '\multirow{3}{*}{\\texttt{GBERT$_{Base}$}}'
gbert2 = '&\\texttt{RP-Crowd-2}'

for key in gbert_models_min2:
    data = pickle.load( open( path+key, "rb" ))
    gbert2 += "&"+ str(round(accuracy_score(data["y_trues"],data["decisions"]),3))
gbert2 +=  "\\\\ \n"

gbert3 = '&\\texttt{RP-Crowd-3}'
for key in gbert_models_min3:
    data = pickle.load( open( path+key, "rb" ))
    gbert3 += "&"+ str(round(accuracy_score(data["y_trues"],data["decisions"]),3))
gbert3 +=  "\\\\\n"

gbertmod = '&\\texttt{RP-Mod}'
for key in gbert_models_mod:
    data = pickle.load( open( path+key, "rb" ))
    gbertmod += "&"+ str(round(accuracy_score(data["y_trues"],data["decisions"]),3))
gbertmod +=  "\\\\ \hline \n"





bottomstring='\\hline \n\
\\end{tabular}'


print(table_header + bertbase + bertbaserp2 + bertbaserp3 +bertbasemod + berthate + berthate2 + berthate3 + berthatemod + gbert + gbert2 + gbert3+ gbertmod+bottomstring)


\begin{tabular}{ llllll }   
\multicolumn{2}{c}{}&\multicolumn{1}{c}{Single-Task}& \multicolumn{3}{ c }{Multi-Task} \\ 
& & & $\alpha$ =0.1 & $\alpha$ = 0.5 & $\alpha$ = 0.9\\ \hline 
\multirow{3}{*}{\texttt{BERT$_{Base}$}}&\texttt{RP-Crowd-2}&0.728&0.789&0.786&0.781\\ 
&\texttt{RP-Crowd-3}&0.806&0.815&0.825&0.823\\ 
&\texttt{RP-Mod}&0.5&0.719&0.711&0.706\\ \hline 
\multirow{3}{*}{\texttt{BERT$_{Hate}$}}&\texttt{RP-Crowd-2}&0.5&0.788&0.782&0.791\\ 
&\texttt{RP-Crowd-3}&0.836&0.798&0.815&0.808\\
&\texttt{RP-Mod}&0.5&0.717&0.707&0.709\\ \hline 
\multirow{3}{*}{\texttt{GBERT$_{Base}$}}&\texttt{RP-Crowd-2}&0.665&0.811&0.811&0.814\\ 
&\texttt{RP-Crowd-3}&0.821&0.833&0.827&0.84\\
&\texttt{RP-Mod}&0.696&0.727&0.727&0.721\\ \hline 
\hline 
\end{tabular}


In [4]:
import pandas as pd
import numpy as np

## F1 TABLE

table_header = '\\begin{tabular}{ llllll }   \n\
\\multicolumn{2}{c}{}&\\multicolumn{1}{c}{Single-Task}& \\multicolumn{3}{ c }{Multi-Task} \\\ \n\
& & & $\\alpha$ =0.1 & $\\alpha$ = 0.5 & $\\alpha$ = 0.9\\\ \\hline \n\
'

## BERT BASE MODEL

bertbase = '\multirow{3}{*}{\\texttt{BERT$_{Base}$}}'
bertbaserp2 = '&\\texttt{RP-Crowd-2}'

for key in bertbase_models_min2:
    data = pickle.load( open( path+key, "rb" ))
    bertbaserp2 += "&"+ str(round(f1_score(data["y_trues"],data["decisions"]),3))
bertbaserp2 +=  "\\\\ \n"

bertbaserp3 = '&\\texttt{RP-Crowd-3}'
for key in bertbase_models_min3:
    data = pickle.load( open( path+key, "rb" ))
    bertbaserp3 += "&"+ str(round(f1_score(data["y_trues"],data["decisions"]),3))
bertbaserp3 +=  "\\\\ \n"


bertbasemod = '&\\texttt{RP-Mod}'
for key in bertbase_models_mod:
    data = pickle.load( open( path+key, "rb" ))
    bertbasemod += "&"+ str(round(f1_score(data["y_trues"],data["decisions"]),3))
bertbasemod +=  "\\\\ \hline \n"

############### BERT HATESPEECH MODEl

berthate = '\multirow{3}{*}{\\texttt{BERT$_{Hate}$}}'
berthate2 = '&\\texttt{RP-Crowd-2}'

for key in berthate_models_min2:
    data = pickle.load( open( path+key, "rb" ))
    berthate2 += "&"+ str(round(f1_score(data["y_trues"],data["decisions"]),3))
berthate2 +=  "\\\\ \n"

berthate3 = '&\\texttt{RP-Crowd-3}'
for key in berthate_models_min3:
    data = pickle.load( open( path+key, "rb" ))
    berthate3 += "&"+ str(round(f1_score(data["y_trues"],data["decisions"]),3))
berthate3 +=  "\\\\\n"

berthatemod = '&\\texttt{RP-Mod}'
for key in berthate_models_mod:
    data = pickle.load( open( path+key, "rb" ))
    berthatemod += "&"+ str(round(f1_score(data["y_trues"],data["decisions"]),3))
berthatemod +=  "\\\\ \hline \n"

############### BERT GBERT MODEl

gbert= '\multirow{3}{*}{\\texttt{GBERT$_{Base}$}}'
gbert2 = '&\\texttt{RP-Crowd-2}'

for key in gbert_models_min2:
    data = pickle.load( open( path+key, "rb" ))
    gbert2 += "&"+ str(round(f1_score(data["y_trues"],data["decisions"]),3))
gbert2 +=  "\\\\ \n"

gbert3 = '&\\texttt{RP-Crowd-3}'
for key in gbert_models_min3:
    data = pickle.load( open( path+key, "rb" ))
    gbert3 += "&"+ str(round(f1_score(data["y_trues"],data["decisions"]),3))
gbert3 +=  "\\\\\n"

gbertmod = '&\\texttt{RP-Mod}'
for key in gbert_models_mod:
    data = pickle.load( open( path+key, "rb" ))
    gbertmod += "&"+ str(round(f1_score(data["y_trues"],data["decisions"]),3))
gbertmod +=  "\\\\ \hline \n"





bottomstring='\\hline \n\
\\end{tabular}'


print(table_header + bertbase + bertbaserp2 + bertbaserp3 +bertbasemod + berthate + berthate2 + berthate3 + berthatemod + gbert + gbert2 + gbert3+ gbertmod+bottomstring)




\begin{tabular}{ llllll }   
\multicolumn{2}{c}{}&\multicolumn{1}{c}{Single-Task}& \multicolumn{3}{ c }{Multi-Task} \\ 
& & & $\alpha$ =0.1 & $\alpha$ = 0.5 & $\alpha$ = 0.9\\ \hline 
\multirow{3}{*}{\texttt{BERT$_{Base}$}}&\texttt{RP-Crowd-2}&0.768&0.794&0.795&0.779\\ 
&\texttt{RP-Crowd-3}&0.804&0.827&0.826&0.825\\ 
&\texttt{RP-Mod}&0.0&0.708&0.7&0.699\\ \hline 
\multirow{3}{*}{\texttt{BERT$_{Hate}$}}&\texttt{RP-Crowd-2}&0.0&0.787&0.771&0.795\\ 
&\texttt{RP-Crowd-3}&0.841&0.805&0.819&0.814\\
&\texttt{RP-Mod}&0.0&0.721&0.721&0.71\\ \hline 
\multirow{3}{*}{\texttt{GBERT$_{Base}$}}&\texttt{RP-Crowd-2}&0.744&0.814&0.809&0.811\\ 
&\texttt{RP-Crowd-3}&0.826&0.833&0.823&0.845\\
&\texttt{RP-Mod}&0.675&0.737&0.732&0.72\\ \hline 
\hline 
\end{tabular}


In [5]:
from sklearn.metrics import roc_auc_score


## AUC TABLE

table_header = '\\begin{tabular}{ llllll }   \n\
\\multicolumn{2}{c}{}&\\multicolumn{1}{c}{Single-Task}& \\multicolumn{3}{ c }{Multi-Task} \\\ \n\
& & & $\\alpha$ =0.1 & $\\alpha$ = 0.5 & $\\alpha$ = 0.9\\\ \\hline \n\
'

## BERT BASE MODEL

bertbase = '\multirow{3}{*}{\\texttt{BERT$_{Base}$}}'
bertbaserp2 = '&\\texttt{RP-Crowd-2}'

for key in bertbase_models_min2:
    data = pickle.load( open( path+key, "rb" ))
    bertbaserp2 += "&"+ str(round(roc_auc_score(data["y_trues"],[x[1] for x in data["probs"]]),3))
bertbaserp2 +=  "\\\\ \n"

bertbaserp3 = '&\\texttt{RP-Crowd-3}'
for key in bertbase_models_min3:
    data = pickle.load( open( path+key, "rb" ))
    bertbaserp3 += "&"+ str(round(roc_auc_score(data["y_trues"],[x[1] for x in data["probs"]]),3))
bertbaserp3 +=  "\\\\ \n"


bertbasemod = '&\\texttt{RP-Mod}'
for key in bertbase_models_mod:
    data = pickle.load( open( path+key, "rb" ))
    bertbasemod += "&"+ str(round(roc_auc_score(data["y_trues"],[x[1] for x in data["probs"]]),3))
bertbasemod +=  "\\\\ \hline \n"

############### BERT HATESPEECH MODEl

berthate = '\multirow{3}{*}{\\texttt{BERT$_{Hate}$}}'
berthate2 = '&\\texttt{RP-Crowd-2}'

for key in berthate_models_min2:
    data = pickle.load( open( path+key, "rb" ))
    berthate2 += "&"+ str(round(roc_auc_score(data["y_trues"],[x[1] for x in data["probs"]]),3))
berthate2 +=  "\\\\ \n"

berthate3 = '&\\texttt{RP-Crowd-3}'
for key in berthate_models_min3:
    data = pickle.load( open( path+key, "rb" ))
    berthate3 += "&"+ str(round(roc_auc_score(data["y_trues"],[x[1] for x in data["probs"]]),3))
berthate3 +=  "\\\\\n"

berthatemod = '&\\texttt{RP-Mod}'
for key in berthate_models_mod:
    data = pickle.load( open( path+key, "rb" ))
    berthatemod += "&"+ str(round(roc_auc_score(data["y_trues"],[x[1] for x in data["probs"]]),3))
berthatemod +=  "\\\\ \hline \n"

############### BERT GBERT MODEl

gbert= '\multirow{3}{*}{\\texttt{GBERT$_{Base}$}}'
gbert2 = '&\\texttt{RP-2}'

for key in gbert_models_min2:
    data = pickle.load( open( path+key, "rb" ))
    gbert2 += "&"+ str(round(roc_auc_score(data["y_trues"],[x[1] for x in data["probs"]]),3))
gbert2 +=  "\\\\ \n"

gbert3 = '&\\texttt{RP-3}'
for key in gbert_models_min3:
    data = pickle.load( open( path+key, "rb" ))
    gbert3 += "&"+ str(round(roc_auc_score(data["y_trues"],[x[1] for x in data["probs"]]),3))
gbert3 +=  "\\\\\n"

gbertmod = '&\\texttt{RP-Mod}'
for key in gbert_models_mod:
    data = pickle.load( open( path+key, "rb" ))
    gbertmod += "&"+ str(round(roc_auc_score(data["y_trues"],[x[1] for x in data["probs"]]),3))
gbertmod +=  "\\\\ \hline \n"





bottomstring='\\hline \n\
\\end{tabular}'


print(table_header + bertbase + bertbaserp2 + bertbaserp3 +bertbasemod + berthate + berthate2 + berthate3 + berthatemod + gbert + gbert2 + gbert3+ gbertmod+bottomstring)




\begin{tabular}{ llllll }   
\multicolumn{2}{c}{}&\multicolumn{1}{c}{Single-Task}& \multicolumn{3}{ c }{Multi-Task} \\ 
& & & $\alpha$ =0.1 & $\alpha$ = 0.5 & $\alpha$ = 0.9\\ \hline 
\multirow{3}{*}{\texttt{BERT$_{Base}$}}&\texttt{RP-Crowd-2}&0.84&0.868&0.869&0.864\\ 
&\texttt{RP-Crowd-3}&0.879&0.899&0.899&0.903\\ 
&\texttt{RP-Mod}&0.498&0.793&0.785&0.774\\ \hline 
\multirow{3}{*}{\texttt{BERT$_{Hate}$}}&\texttt{RP-Crowd-2}&0.499&0.867&0.869&0.871\\ 
&\texttt{RP-Crowd-3}&0.906&0.888&0.898&0.894\\
&\texttt{RP-Mod}&0.5&0.785&0.774&0.774\\ \hline 
\multirow{3}{*}{\texttt{GBERT$_{Base}$}}&\texttt{RP-2}&0.766&0.892&0.889&0.889\\ 
&\texttt{RP-3}&0.899&0.914&0.912&0.914\\
&\texttt{RP-Mod}&0.77&0.79&0.791&0.791\\ \hline 
\hline 
\end{tabular}
